# hdpws Example Jupyter Notebook
![SPASE inside](https://spase-group.org/assets/images/spase-inside.png)
This [Jupyter notebook](https://jupyter.org/) demonstrates using the [hdpws](https://pypi.org/project/hdpws/) Python package to access [Space Physics Archive Search and Extract](https://spase-group.org/) (SPASE) metadata documents from the [Heliophysics Data Portal](https://heliophysicsdata.gsfc.nasa.gov/) (HDP).  It assumes some familarity with the [SPASE data model](https://spase-group.org/data/index.html).  This notebook contains the following sections:
1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Get MeasurementTypes](#Get-MeasurementTypes)
4. [Get SpectralRanges](#Get-SpectralRanges)
5. [Get PhenomenonTypes](#Get-PhenomenonTypes)
6. [Get ObservedRegions](#Get-ObservedRegions)
7. [Get ObservatoryIDs](#Get-ObservatoryIDs)
8. [Get Numerical/Display Data](#Get-Numerical/Display-Data)
9. [Get Catalog](#Get-Catalog)
10. [Get Collection](#Get-Collection)
11. [Get Document](#Get-Document)
12. [Get a SPASE document by ResourceID](#Get-a-SPASE-document-by-ResourceID)
13. [Conditionally get a SPASE document](#Conditionally-get-a-SPASE-document)
14. [Error Handling](#Error-Handling)
15. [Additional Documentation](#Additional-Documentation)

## Prerequisites
Install the prerequisite software from [Python Package Index](https://pypi.org/project/hdpws/) (PyPI) software repository.
1. pip install hdpws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [14]:
from hdpws.hdpws import HdpWs
from hdpws import NAMESPACES as NS
from hdpws.resourcetype import ResourceType as rt
from hdpws.spase import AccessURL, HapiAccessURL

from IPython.core.display import HTML

# disable warnings about vspo-dev certificate
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

hdp = HdpWs(endpoint='http://ec2-44-212-210-55.compute-1.amazonaws.com/WS/hdp/1/',
           disable_ssl_certificate_validation=True)


## Get MeasurementTypes
The following code demonstrates how to get the list of available /Spase/MeasurementType values.

In [15]:
result = hdp.get_measurement_types()
print('HDP MeasurementTypes:')
for value in result['MeasurementType']:
    print(f'    {value}')


HDP MeasurementTypes:
    ActivityIndex
    Dopplergram
    Dust
    ElectricField
    EnergeticParticles
    Ephemeris
    ImageIntensity
    InstrumentStatus
    IonComposition
    Irradiance
    MagneticField
    Magnetogram
    NeutralAtomImages
    NeutralGas
    Profile
    Radiance
    Spectrum
    SPICE
    ThermalPlasma
    Waves
    Waves.Active
    Waves.Passive


## Get SpectralRanges
The following code demonstrates how to get the list of available /Spase/SpectralRange values.

In [16]:
result = hdp.get_spectral_ranges()
print('HDP SpectralRanges:')
for value in result['SpectralRange']:
    print(f'    {value}')

HDP SpectralRanges:
    CaK
    ExtremeUltraviolet
    FarUltraviolet
    GammaRays
    Halpha
    HardXrays
    He10830
    He304
    Infrared
    K7699
    LBHBand
    Microwave
    NaD
    Ni6768
    Optical
    RadioFrequency
    SoftXRays
    Ultraviolet
    WhiteLight
    XRays


## Get PhenomenonTypes
The following code demonstrates how to get the list of available /Spase/PhenomenonType values.

In [17]:
result = hdp.get_phenomenon_types()
print('HDP PhenomenonTypes:')
for value in result['PhenomenonType']:
    print(f'    {value}')

HDP PhenomenonTypes:
    ActiveRegion
    Aurora
    BowShockCrossing
    CoronalHole
    CoronalMassEjection
    EITWave
    EnergeticSolarParticleEvent
    ForbushDecrease
    GeomagneticStorm
    InterplanetaryShock
    MagneticCloud
    MagnetopauseCrossing
    RadioBurst
    SectorBoundaryCrossing
    SolarFlare
    SolarWindExtreme
    StreamInteractionRegion
    Substorm


## Get ObservedRegions
The following code demonstrates how to get the list of available /Spase/ObservedRegion values.

In [18]:
result = hdp.get_observed_regions()
observed_regions = result['ObservedRegion']
print(f'{len(observed_regions)} HDP Observed Regions:')
for value in observed_regions[0:9]:
    print(f'    {value}')
print('    ...')

122 HDP Observed Regions:
    Asteroid
    Comet
    Earth
    Earth.Magnetosheath
    Earth.Magnetosphere
    Earth.Magnetosphere.Magnetotail
    Earth.Magnetosphere.Main
    Earth.Magnetosphere.Plasmasphere
    Earth.Magnetosphere.Polar
    ...


## Get ObservatoryIDs
The following code demonstrates how to get the list of available /Spase/Observatory/ResourceID values.

In [19]:
result = hdp.get_observatory_ids()
observatory_ids = result['ObservatoryID']
print(f'{len(observatory_ids)} HDP ObservatoryIDs:')
for value in observatory_ids[0:9]:
    print(f'    {value}')
print('    ...')

2003 HDP ObservatoryIDs:
    spase://SMWG/Observatory/JSPO
    spase://SMWG/Observatory/MEASURE/Darksky.Observatory
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-EPO/BMLS
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-GBO/GBAY
    spase://SMWG/Observatory/THEMIS/E
    spase://SMWG/Observatory/Ground/Karmoy
    spase://SMWG/Observatory/Ground/Val.Joyeux
    spase://SMWG/Observatory/Ground/Pangnirtung
    spase://SMWG/Observatory/Ground/Kevo
    ...


## Get Numerical/Display Data
The following code demonstrates how to get SPASE Numerical/Display data documents matching the specified search criteria.

In [20]:
query = {
    'InstrumentID': 'spase://SMWG/Instrument/ACE/CRIS',
    'Cadence': '=PT1H',
    'ObservedRegion': 'Heliosphere.NearEarth',
    'MeasurementType': 'EnergeticParticles',
    'AccessRights': 'Open',
    'Format': 'CDF'
}       
types = [rt.NUMERICAL_DATA, rt.DISPLAY_DATA]
time_range = ['2022-01-01', '2022-01-02']
result = hdp.get_spase_data(types, query, time_range)
if result['HttpStatus'] == 200:
    print('Results:')
    for spase in result['Result'].findall('.//Spase', namespaces=NS):
        id = spase.find('.//ResourceID', namespaces=NS)
        name = spase.find('.//Name', namespaces=NS)
        description = spase.find('.//Description', namespaces=NS)
        #print(name.text)
        print('ResourceID: ', id.text)
        print('Description: ', description.text[:60], '...')
        display(HTML('<a href="' + hdp.get_spase_url(id.text) +
                     '" target="_blank">' + 
                    'HTML representation of SPASE</a>'))
        
        ws_access_url_element = \
            spase.find('.//AccessURL[Style="WebService"]', namespaces=NS)
        if ws_access_url_element is not None:
            ws_access_url = AccessURL(ws_access_url_element)
            print('Web Service access:')
            display(HTML('&nbsp;&nbsp;&nbsp;&nbsp;<a href="' + 
                         ws_access_url.url + '" target="_blank">' + 
                         'Example data access code</a>'))
            #print('    URL: ', ws_access_url.url)
            
        hapi_access_url_element = \
            spase.find('.//AccessURL[Style="HAPI"]', namespaces=NS)
        if hapi_access_url_element is not None:
            hapi_access_url = HapiAccessURL(hapi_access_url_element)
            print('HAPI access:')
            print('    URL: ', hapi_access_url.url)
            print('    Dataset IDs: ', hapi_access_url.product_key)
            hapi_display_url = hapi_access_url.get_html_url()
            hapi_html_link = "&nbsp;&nbsp;&nbsp;&nbsp;<a href=""" + \
                hapi_access_url.get_html_url() + \
                """ target=""_blank"">HAPI access information</a>"""
            display(HTML(hapi_html_link))
        print('=======================================')

Results:
ResourceID:  spase://NASA/NumericalData/ACE/CRIS/L2/PT1H
Description:  ACE Cosmic Ray Isotope Spectrometer (CRIS) intensities and c ...


Web Service access:


HAPI access:
    URL:  https://cdaweb.gsfc.nasa.gov/hapi
    Dataset IDs:  ['AC_H2_CRIS']


ResourceID:  spase://NASA/NumericalData/ACE/CRIS/L2/P1D
Description:  ACE Cosmic Ray Isotope Spectrometer (CRIS) intensities and c ...


Web Service access:


HAPI access:
    URL:  https://cdaweb.gsfc.nasa.gov/hapi
    Dataset IDs:  ['AC_H3_CRIS']


## Get Catalog
The following code demonstrates how to get a SPASE Catalog document matching the specified criteria.

In [21]:
query = { 
    'InstrumentID': 'spase://SMWG/Instrument/ACE/MAG',
    'PhenomenonType': 'MagneticCloud',
    'Description': 'ICME'
}   
time_range = ['1999-01-01', '1999-01-02']
result = hdp.get_spase_catalog(query, time_range)
if result['HttpStatus'] == 200:
    print('Result Catalogs:')
    for catalog in result['Result'].findall('.//Catalog',
                                           namespaces=NS):
        print('ResourceID: ', catalog.findall('.//ResourceID',
                             namespaces=NS)[0].text)
        print('ResourceName: ', catalog.findall('.//ResourceName',
                                     namespaces=NS)[0].text) 
        print('Description:', 
              catalog.findall('.//Description',
                              namespaces=NS)[0].text[:750],
             '...')

Result Catalogs:
ResourceID:  spase://NASA/Catalog/CME-ICME_Cho2003
ResourceName:  CME/ICME list by Cho et al., 2003
Description: CME/ICME list created by Cho et al., 2003. Method:

CMEs are selected which have temporal and spatial proximity to the type II events in Table 1 of Fry et al (2003). C2 appearance time, PA, and speed from SOHO/LASCO CME catalog are used (http://cdaw.gsfc.nasa.gov/CME_list/). They say: "The procedure for examining the arrival time predictions of ICMEs and IP shocks for the near-simultaneous events are summarized as follows: 

(1) From the 173 type II events of Fry et al. [2003], we choose a total of 101 CMEs that are within a threshold window (~90 min). 
(2) We select 89 events from this group by comparing the position angles and the coordinate information of the associated flares. 
(3) We apply the adopted prediction models (the ensemble of ...


## Get Collection
The following code demonstrates how to get a SPASE Collection document matching the specified criteria.

In [22]:
query = { 
    'ResourceID': 'spase://NASA/Collection/IRIS_AIA',
    'MemberID': 'spase://NASA/NumericalData/SDO/AIA/PT10S',
    'Description': 'IRIS AND SDO and AIA'
}   
result = hdp.get_spase_collection(query)
if result['HttpStatus'] == 200:
    for collection in result['Result'].findall('.//Collection',
                                               namespaces=NS):
        print('ResourceID: ', collection.find('.//ResourceID',
                                              namespaces=NS).text)
        print('ResourceName: ', collection.find('.//ResourceName',
                                                namespaces=NS).text)
        print('Collection members:')
        for member in collection.findall('.//Member',
                                         namespaces=NS):
            member_id = member.find('.//MemberID',
                                    namespaces=NS)
            print('    ', member_id.text)
                

ResourceID:  spase://NASA/Collection/IRIS_AIA
ResourceName:  Co-aligned Interface Region Imaging Spectrograph (IRIS) and Solar Dynamics Observatory (SDO) Atmospheric Imaging Assembly (AIA) Observations
Collection members:
     spase://NASA/NumericalData/IRIS/IRIS/PT1S
     spase://NASA/NumericalData/SDO/AIA/PT10S


## Get Document
The following code demonstrates how to get the SPASE Document matching the specified criteria.

In [23]:
query = {
    'ResourceID': 'spase://SMWG/Document/HPDE/Policy/HP_DataPolicy_v1.2',
    'DOI': '10.21978/P83P78',
    'Description': '"Program Data Management Plan"'
}
result = hdp.get_spase_document(query)
if result['HttpStatus'] == 200:
    print('Result Documents:')
    for document in result['Result'].findall('.//Document', 
                                             namespaces=NS):
        print('ResourceName: ', document.find('.//ResourceName', 
                                              namespaces=NS).text)
        description = document.find('.//Description', 
                                    namespaces=NS)
        print('Description: ', description.text)
        url = document.find('.//AccessInformation/AccessURL/URL',
                           namespaces=NS)
        print('URL: ', url.text)

Result Documents:
ResourceName:  NASA Heliophysics Science Data Management Policy v. 1.2
Description:  NASA Heliophysics document that provides guidance on all aspects of HP data, including what needs to go into a Program Data Management Plan, the structure and use of archives, required formats, and a description of the data registry, all in the context of the data lifecycle.
URL:  https://cdaweb.gsfc.nasa.gov/pub/documents/HPDE/Heliophysics_Data_Policy_v1.2_2016Oct04_signed.pdf


## Get a SPASE document by ResourceID
The following code demonstrates how to get the SPASE document with the specified ResourceID.

In [24]:
resource_ids = [
    'spase://NASA/Collection/IRIS_AIA',
    'spase://SMWG/Service/CCMC/Models'
]
result = hdp.get_spase(resource_ids)
if result['HttpStatus'] == 200:
    print('get_spase Result ResourceIDs:')
    for spase in result['Result'].findall('.//Spase',
                                         namespaces=NS):
        print('ResourceID: ', spase.find('.//ResourceID',
                           namespaces=NS).text)
        print('ResourceName: ', spase.find('.//ResourceName',
                                   namespaces=NS).text)
    if 'Last-Modified' in result:
        last_modified = result['Last-Modified']
        print('last_modified = ', last_modified.isoformat())

get_spase Result ResourceIDs:
ResourceID:  spase://SMWG/Service/CCMC/Models
ResourceName:  CCMC Modelweb page
ResourceID:  spase://SMWG/Service/CCMC/Models
ResourceName:  CCMC Modelweb page
ResourceID:  spase://NASA/Collection/IRIS_AIA
ResourceName:  Co-aligned Interface Region Imaging Spectrograph (IRIS) and Solar Dynamics Observatory (SDO) Atmospheric Imaging Assembly (AIA) Observations
last_modified =  2019-05-01T14:06:02+00:00


## Conditionally get a SPASE document
The following code demonstrates how to get a SPASE document only if it has been modified since the last time it was gotten.

In [25]:
if last_modified is not None:
    result = hdp.get_spase(resource_ids, if_modified_since=last_modified)

    if result['HttpStatus'] == 304:
        print('get_spase if_modified_since ', last_modified.isoformat(),
                  'return Not Modified')
    else:
        print('get_spase if_modified_since returned an unexpected result',
             result['HttpStatus'])

get_spase if_modified_since  2019-05-01T14:06:02+00:00 return Not Modified


## Error Handling
The following code demonstrates how to view an error message from the server in response to a bad request from the client.

In [26]:
query = {
    'ResourceID': ['spase://NASA/NumericalData/ACE/CRIS/L2/P1D'],
    'Description': '~bad'
}
result = hdp.get_spase_data(types, query, time_range)
if result['HttpStatus'] != 200:
    print(f'hdp.get_spase() failed with status = {result["HttpStatus"]}')
    if 'ErrorMessage' in result:
        print(f'ErrorMessage = {result["ErrorMessage"]}')
        print(f'ErrorDescription = {result["ErrorDescription"]}')
    #else:
        #print(f'HttpText = {result["ErrorText"]}')
        print('Display of actual HTML page returned:')
        display(HTML(result['ErrorText']))

hdp.get_spase() failed with status = 400
ErrorMessage = Bad Request
ErrorDescription = Invalid Description syntax.
Display of actual HTML page returned:


## Additional Documentation
View the [hdpws API documentation](https://heliophysicsdata.gsfc.nasa.gov/WebServices/py/hdpws/) for a description of additional features.